In [1]:
# Lab 7 - Text generation with LSTM
#
# Step 1 (not assessed): build and train a model to generate text in the style of a corpus.
#
# Based on the Keras text generation example (https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py)
#
# Step 2: build a model to distinguish genuine from fake sentences.

In [2]:
# Import essential modules
import pickle
import random
import sys
import time
import keras

import numpy as np
from sklearn.model_selection import train_test_split

from keras.layers import Input, LSTM, GRU, Dense, Activation, Conv1D, Dropout, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM
from keras.regularizers import l1, l2
from keras.optimizers import RMSprop, Adam, Nadam, SGD
from keras.models import Model, Sequential
from keras.models import save_model
from keras.utils.data_utils import get_file
from keras.layers.advanced_activations import LeakyReLU
from keras import initializers

from IPython.display import clear_output

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [3]:
# Helper function to sample an index from an array of predictions.
#
# The input array 'preds' should be the output of a text generation model.
# The elements contain the values of the units in the final layer.
# Each unit corresponds to a character in the text alphabet.
# The final layer should have SoftMax activation, and thus the
# value corresponds to the 'strength of prediction' of that character
# as the next output value---so the maximum value indicates which character
# is most strongly predicted (considerd most likely) as the next one.
#
def sample(preds, temperature=1.0):
    # Convert to high-precision datatype (we are going to be manipulating some
    # very small values in this function)
    preds = np.asarray(preds).astype('float64')  
    
    # The next line has the effect of raising each prediction value to the power 1/T.
    # It's done using logs to improve numerical precision.  This is a kind of value-dependent
    # scaling: for T < 1.0 (1/T > 1.0), small values are made smaller (proportionally) than 
    # large values (unlike a linear scaling, such as multiplication by 0.9, which scales all values
    # the same).
    #
    # Example: Consider that we have only two symbols (letters) in our alphabet, and our 
    # probabilities are [0.2, 0.8].  A temperature of 1.0 means 'do not adjust the
    # probabilities at all', so in this case there will be a 20% chance that the 
    # function will return 'symbol 0' and an 80% chance  that it will return 'symbol 1'.
    # Note that symbol 1 is 4x more likely than symbol 0.
    #
    # Now: if we supply a temperature of 0.5, our probabilites will be raised to the
    # power 1/0.5 = 2, becoming [0.04, 0.64].  These will then be normalized to sum to 1,
    # but anyway it is clear that symbol 1 is here 16x (the square of 4x) more likely than 
    # symbol 0.
    #
    # Conversely, for a temperature of 2, our probabilities will be raised to 0.5 (square-rooted),
    # becoming [.4472, 0.8944] - and so here symbol 1 is only 2x (sqrt of 4x) more likely than
    # symbol 0.
    #
    # So: low temperatures make the distribution peakier, exaggerating the difference between
    # values.  High temperatures flatten the distribution, reducing the difference between values.
    #
    # As the return value is a sample of the manipulated distribution, manipulating it to
    # be peakier (by supplying a low temperature) makes the sample more conservative, i.e.
    # more likely to pick the highest-probability symbol.
    #
    # Making the distribution flatter (by suppyling a high temperature) causes the
    # sample to be less conservative, i.e. more likely to pick some lower-likelihood
    # symbol.
    #
    # Phew!
    preds = np.exp(np.log(preds) / temperature)
    
    preds = preds / np.sum(preds)  # ensure that probs sum to 1
    probas = np.random.multinomial(1, preds, 1)  # take 1 sample from the distribution
    return np.argmax(probas)

In [61]:
# Decide how much data to use for training.
# You might want to reduce this to ~100k for faster experimentation, and then bring it back
# to 600k when you're happy with your network architecture.
# IMPORTANT: mke sure you end up with a 57-symbol alphabet after reducing the corpus size!
# If the number of symbols (shown in the next cell) gets smaller than it was with the full
# corpus, bring your sample size back up.  This is necessary because the encoding used for
# training must match that used for assessment.
#desired_num_chars = 600*1000  # Max: 600893
desired_num_chars = 480139  # Max: 600893

random.seed(43)  # Fix random seed for repeatable results.

# Slurp down all of Nietzsche from Amazon.
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('original corpus length:', len(text))

start_index = random.randint(0, len(text) - desired_num_chars - 1)
text = text[start_index:start_index + desired_num_chars]
text
print('length for training:', len(text))

original corpus length: 600901
length for training: 480139


In [62]:
# Let's have a quick look at a random exceprt.
#
# Caution: Nietzsche might drive you mad: dare you behold more than 1000 of his terrible chars..? 
sample_length = 1000

random.seed(None)  # Seeds random from current time (so re-eval this cell for a new sample).

start_index = random.randint(0, len(text) - sample_length - 1)
print(text[start_index:start_index+sample_length])

tle stone; all the greater need is there of
a courage that is not ashamed of such humble labor and that will oppose
persistence, to all contempt. it is, finally, also true that countless
single observations concerning the human, all-too-human, have been
first made and uttered in circles accustomed, not to furnish matter for
scientific knowledge, but for intellectual pleasure-seeking; and the
original home atmosphere--a very seductive atmosphere--of the moral
maxim has almost inextricably interpenetrated the entire species, so
that the scientific man involuntarily manifests a sort of mistrust of
this species and of its seriousness. but it is sufficient to point to
the consequences: for already it is becoming evident that events of the
most portentous nature are developing in the domain of psychological
observation. what is the leading conclusion arrived at by one of the
subtlest and calmest of thinkers, the author of the work "concerning the
origin of the moral feelings", as a result of

In [63]:
# Establish the alphabet (set of symbols) we are going to use.
chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars)

char_indices = dict((c, i) for i, c in enumerate(chars))  # Map to look up index of a particular char (e.g. x['a'] = 0)
indices_char = dict((i, c) for i, c in enumerate(chars))  # Map to look up char at an index (e.g. x[0] = 'a')

total chars: 57
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¤', '¦', '«', 'ã']


In [64]:
# Establish a training set of semi-redundant (i.e. overlapping) sequences of maxlen characters.
maxlen = 40
step = 3
sentences = []  # Not syntactic sentences, but just sequences of 40 chars pulled from the corpus.
next_chars = [] # next_chars[n] stores the character which followed sentences[n]
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 160033


In [65]:
# Convert the data to one-hot encoding.
# 'x' will contain the one-hot encoding of the training 'sentences'.
# 'y' will contain the one-hot encoding of the 'next char' for each sentence.
#
# 
# Let's consider that we have N sentences of length L:
#
# The 'native' encoding is an NxL matrix where element [n][l]
# is the symbol index for character at index (l) of sentence (n)
# (e.g., say, 5, corresponding to 'e').
#
# The one-hot encoding is an NxLxS matrix, where S is the 
# number of symbols in the alphabet, such that element [n][l][s]
# is 1 if the character at index (l) in sentence (n) has the
# symbol index (s), and 0 otherwise.
def onehot_encode(sentence, maxlen):
    x = np.zeros((maxlen, len(chars)), dtype=np.bool)
    for t, char in enumerate(sentence):
        x[t, char_indices[char]] = 1
    return x

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    x[i,:,:] = onehot_encode(sentence, maxlen)
    y[i, :] = onehot_encode(next_chars[i], 1)

print(x.shape)
print(y.shape)

(160033, 40, 57)
(160033, 57)


In [66]:
# Build the generator model: a single GRU layer with 128 cells.
generator_model = Sequential()
generator_model.add(GRU(128, input_shape=(maxlen, len(chars))))
generator_model.add(Dense(len(chars)))
generator_model.add(Activation('softmax'))

# You could experiment with NAdam instead of RMSProp.
optimizer = RMSprop(lr=0.01)
generator_model.compile(loss='categorical_crossentropy', optimizer=optimizer)
trained_epochs = 0

In [67]:
def generate_sentence_list(seed_list, length=400, temperature=0.25):
    sentence_list_1 = [];
    sentence_list_2 = [];
    sentence_list_3 = [];
    generated_list_1 = [];
    generated_list_2 = [];
    generated_list_3 = [];
    n = len(seed_list)
    # copy lists
    temperature_1 = temperature
    temperature_2 = (temperature + 0.005)
    temperature_3 = max(0.01, (temperature - 0.005))
    for seed in seed_list:
        sentence_list_1.append(seed[:])
        sentence_list_2.append(seed[:])
        sentence_list_3.append(seed[:])
        generated_list_1.append(seed[:])
        generated_list_2.append(seed[:]) 
        generated_list_3.append(seed[:]) 
    
    for i in range(length):
      
        workdone = (i+1)*1.0 / length
        sys.stdout.write("\rgenerating sentences: [{0:20s}] {1:.1f}%".format('#' * int(workdone * 20), workdone*100))
        sys.stdout.flush()
            
        x_pred_list = np.zeros((n, maxlen, len(chars)))
        for j, sentence in enumerate(sentence_list_1):
            for t, char in enumerate(sentence):
                x_pred_list[j, t, char_indices[char]] = 1.

        start = time.time()
        pred_list = generator_model.predict(x_pred_list, verbose=0)
        end = time.time()

        for j in range(n):
            next_index_1 = sample(pred_list[j,:], temperature_1)
            next_char_1 = indices_char[next_index_1]
            generated_list_1[j] += next_char_1
            sentence_list_1[j] = sentence_list_1[j][1:] + next_char_1
            next_index_2 = sample(pred_list[j,:], temperature_2)
            next_char_2 = indices_char[next_index_2]
            generated_list_2[j] += next_char_2
            sentence_list_2[j] = sentence_list_2[j][1:] + next_char_2
            next_index_3 = sample(pred_list[j,:], temperature_3)
            next_char_3 = indices_char[next_index_3]
            generated_list_3[j] += next_char_3
            sentence_list_3[j] = sentence_list_3[j][1:] + next_char_3
    
    sys.stdout.write(' - done\n')
    sys.stdout.flush()
    
    generated_list = generated_list_1 + generated_list_2 + generated_list_3
    return generated_list

def print_sentences(seeds, sentences):
    for seed, sentence in zip(seeds, sentences):
        print('-'*5)
        sys.stdout.write('\x1b[32m')
        sys.stdout.write(sentence[0:len(seed)])
        sys.stdout.write('\x1b[34m')
        sys.stdout.write(sentence[len(seed):-1])
        sys.stdout.write('\x1b[m')
        sys.stdout.write('\n')    
        sys.stdout.flush()
        
def pick_sentences(n, maxlen):
    global text    
    start_index_list = np.random.randint(len(text) - maxlen - 1, size=(1, n)).flatten().tolist()
    seed_list = [] 
    for start_index in start_index_list:
        seed_list.append(text[start_index: start_index + maxlen])
    return seed_list

In [68]:
# Generate 3 seeds which we will use to inspect the progress of our training:
#preview_seeds = pick_sentences(3, maxlen=40)

# Train the model, output generated text after each iteration
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    generator_model.fit(x, y,
                  batch_size=1024,
                  epochs=4)

    #generated_sentences = generate_sentence_list(preview_seeds)
    #print_sentences(preview_seeds, generated_sentences)


--------------------------------------------------
Iteration 1
Epoch 1/4
160033/160033 [==============================] - 45s 284us/step - loss: 2.28190s - loss: 2.2
Epoch 2/4
160033/160033 [==============================] - 15s 91us/step - loss: 1.8151: 0s - loss: 1.81
Epoch 3/4
160033/160033 [==============================] - 14s 88us/step - loss: 1.6689
Epoch 4/4
160033/160033 [==============================] - 14s 90us/step - loss: 1.5860

--------------------------------------------------
Iteration 2
Epoch 1/4
160033/160033 [==============================] - 15s 92us/step - loss: 1.5336
Epoch 2/4
160033/160033 [==============================] - 19s 121us/step - loss: 1.4971
Epoch 3/4
160033/160033 [==============================] - 18s 110us/step - loss: 1.4689
Epoch 4/4
160033/160033 [==============================] - 15s 94us/step - loss: 1.4466

--------------------------------------------------
Iteration 3
Epoch 1/4
160033/160033 [==============================] - 17s 103us/s

In [69]:
# For a more complete inspection, print out a load of sentences:
#
num_sentences = 100             # how many to generate
sentence_length = 40            # 100--400 is good
sample_temperature = 0.25       # see discussion of temperature up near the top
#sample_temperature = 0.1

start_index_list = np.random.randint(len(text) - maxlen - 1, size=(1, num_sentences)).flatten().tolist()
preview_seeds = [] 
for start_index in start_index_list:
    preview_seeds.append(text[start_index: start_index + maxlen])

generated_sentences = generate_sentence_list(preview_seeds, length=sentence_length, temperature=sample_temperature); 
print_sentences(preview_seeds, generated_sentences)

generating sentences: [####################] 100.0% - done
-----
ed
to place the greatness of man, the commonism of the species of the soul, whi
-----
d, the
genius of the race overflowing from the desires, and in all the stronger
-----
ed at, it is always well (and wise) to finest and profound the will to be under
-----
 marksmen who again and again hit the blindicists and speaking that is a desire
-----
exceptional cases of
wise old codgers and spirit, as a master, or the sense of 
-----
hilosophers on the old
conception of the astiminal spirit, as a master and astu
-----
 defined and designated according to
its of the species of a caster and asturin
-----
 to presuppose two centuries of music
ascestations, as a kindistic preservite a
-----
entional and as a charm, that one should be involonce about the fastionours of 
-----
 the knowledge but the welfare of human himself, and with stronger and invent a
-----
ysiokolax was a popular
name for an actors and species, as a master as a mo

-----
kes prominent, and falsifies for itself
because in the sense of the passive as 
-----
kind will reveal that it is by no means
of the involvest of the same spirit, as
-----
eded in
severing the whole of the last gradity and speaking and history as a gr
-----
f supreme justice, the art of commanding and desires, and a man as a master as 
-----
es not believe
that a philosopher--supposinized and preservituation of the will
-----
idable
filth of all political agitation.

133. in this passion as a singles and
-----
refore it is true.

[17] sache, thing but in all oneself as the same stronger a
-----
y day: and the dream-fancy does exactly and the deniarest and spirit, as in thi
-----
endation for a defect.
(as the opposite wished as a kindiousical asturing and s
-----
nd whatever these honest advocates of "moralizaning morality of "standing of th
-----
tant_, as, for example, our notions of the highest and preservite and species o
-----
ty before god," have hitherto swayed
the species

In [70]:
# This is just a checkpoint, which will let you download and re-upload (or add to git) this model.
save_model(generator_model, './generator_model.h5')

In [71]:
# Generating the training fake sentences for the Discriminator network
#
# These are saved to the file 'fake.pkl' -- you could download this to your
# user drive and re-upload it in a subsequent session, to save regenerating
# it again (in which case you don't need to evaluate this cell).

#training_seeds = pick_sentences(3000, maxlen=40)
training_seeds = pick_sentences(5000, maxlen=40)
training_generated_sentences = generate_sentence_list(training_seeds, length=40)
# Strip out the initial 40 chars (the seed sequence, which is genuine data from the corpus).
for i, sentence in enumerate(training_generated_sentences):
    training_generated_sentences[i] = sentence[40:40+40]
    
output = open('fake.pkl', 'wb')
pickle.dump(training_seeds, output)
pickle.dump(training_generated_sentences, output)
output.close()

generating sentences: [####################] 100.0% - done


In [72]:
# Load the training set from the file
pkl_file = open('fake.pkl', 'rb')
training_seeds = pickle.load(pkl_file)
training_generated_sentences = pickle.load(pkl_file)
pkl_file.close()

In [73]:
# Make a 50:50 set of 'fake' (generated) and genuine sentences:
num_generated = len(training_generated_sentences)
print('Num generated: ', num_generated)
training_real_sentences = pick_sentences(num_generated, maxlen=40)

all_training_sentences = training_generated_sentences + training_real_sentences

n = len(all_training_sentences)
x = np.zeros((n, 40, len(chars)))
y = np.zeros((n, 1))
print('All training sequences: ', n)

for i, sentence in enumerate(all_training_sentences):
    x[i, :, :] = onehot_encode(sentence, maxlen=40)
y[num_generated:] = 1  # Encodes the fact that sentences with indexes larger than (num_generated) are real.


Num generated:  15000
All training sequences:  30000


In [74]:
class PlotLossAccuracy(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.acc = []
        self.losses = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(int(self.i))
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        
        self.i += 1
        
        clear_output(wait=True)
        plt.figure(figsize=(16, 6))
        plt.plot([1, 2])
        plt.subplot(121) 
        plt.plot(self.x, self.losses, label="train loss")
        plt.plot(self.x, self.val_losses, label="validation loss")
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.title('Model Loss')
        plt.legend()
        plt.subplot(122)         
        plt.plot(self.x, self.acc, label="training accuracy")
        plt.plot(self.x, self.val_acc, label="validation accuracy")
        plt.legend()
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.title('Model Accuracy')
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.show();

In [170]:
print('Build model...')

# Define some layers here..

# Use your layers to create the model.
discriminator_model = Sequential()
#discriminator_model.add(LSTM(128, dropout=0.6, recurrent_dropout=0.7,
#                            input_shape=(maxlen, len(chars))))
#discriminator_model.add(LSTM(256, dropout=0.0, recurrent_dropout=0.01,
#                             kernel_regularizer=l2(0.2),
#                             input_shape=(maxlen, len(chars))))

#inputs = Input(shape=(40, 59))
#h = LSTM(256)(inputs)
#h = Dropout(0.2)(h)
#h = Dense(1024, activation='relu')(h)
#h = LSTM(256, return_sequences=False)(h)
#h = Dense(512, activation='tanh')(h)

#output = Dense(1, activation='softmax')(h)

discriminator_model = Sequential()
#discriminator_model.add(Embedding(4020, 40, input_length=59))
discriminator_model.add(LSTM(128, dropout = 0.2, return_sequences=False, 
                             recurrent_dropout = 0.2, input_shape=(maxlen, len(chars)), unit_forget_bias=True,
                             kernel_regularizer=l1(0.01), recurrent_regularizer=l2(0.01)))
#discriminator_model.add(LSTM(256, recurrent_dropout=0.0, return_sequences=False, input_shape=(maxlen, len(chars))))
#discriminator_model.add(LSTM(128, return_sequences=True))
#discriminator_model.add(Conv1D(64, 5, activation='relu', padding='valid', input_shape=(maxlen, len(chars))))
#discriminator_model.add(Conv1D(32, 3, activation='tanh', padding='same'))
#discriminator_model.add(Dropout(0.7))
#discriminator_model.add(Dense(len(chars), activation='relu'))
#discriminator_model.add(LSTM(128, return_sequences=False))
#discriminator_model.add(Flatten())
#discriminator_model.add(Dropout(0.7))
#discriminator_model.add(LSTM(256, dropout = 0.5, return_sequences=True, recurrent_dropout = 0.2))
#discriminator_model.add(GRU(64))
#discriminator_model.add(Flatten())
#discriminator_model.add(Dense(len(chars), activation='relu'))
#discriminator_model.add(Dense(1, activation='sigmoid'))
#discriminator_model.add(Dense(1024))
#discriminator_model.add(LeakyReLU(0.2))
#discriminator_model.add(Dense(512))
#discriminator_model.add(LeakyReLU(0.2))
discriminator_model.add(Dropout(0.4))
#discriminator_model.add(Dense(1))

discriminator_model.add(Dense(1, activation='sigmoid'))
opt = RMSprop(lr=0.01)

# Setup the optimisation strategy.
discriminator_model.compile(optimizer='Adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
                             
print('compiled.')
discriminator_model.summary()

Build model...
compiled.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 128)               95232     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 129       
Total params: 95,361
Trainable params: 95,361
Non-trainable params: 0
_________________________________________________________________


In [171]:
[x_train, x_test, y_train, y_test] = train_test_split(x, y, test_size=0.25, random_state=42)
discriminator_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=4, batch_size=64)
pltCallBack = PlotLossAccuracy()
#discriminator_model.fit(x_train, y_train, 
#                        validation_data=(x_test, y_test), 
#                        epochs=3, batch_size=64,
#                        callbacks=[pltCallBack])

Train on 22500 samples, validate on 7500 samples
Epoch 1/4
22500/22500 [==============================] - 50s 2ms/step - loss: 2.3440 - acc: 0.5101 - val_loss: 0.7294 - val_acc: 0.4945
Epoch 2/4
22500/22500 [==============================] - 43s 2ms/step - loss: 0.7290 - acc: 0.5012 - val_loss: 0.7291 - val_acc: 0.5055
Epoch 3/4
22500/22500 [==============================] - 43s 2ms/step - loss: 0.7286 - acc: 0.4985 - val_loss: 0.7280 - val_acc: 0.5055
Epoch 4/4
22500/22500 [==============================] - 43s 2ms/step - loss: 0.7282 - acc: 0.5026 - val_loss: 0.7282 - val_acc: 0.4945


In [172]:
# Once you're happy with your discriminator model, evaluate this cell to save it:
save_model(discriminator_model, './discriminator_model.h5')
# Run these commands in the terminal to submit your model for assessment.
# git add lab-07/discriminator_model.h5
# git commit -m "Add/update discriminator model."
# git push
# submit-lab 7

score,train = discriminator_model.evaluate(x_train, y_train, batch_size = 64, verbose=0)
score,acc = discriminator_model.evaluate(x_test, y_test, batch_size = 64, verbose=0)
print("Score: %.2f" % (score))
print("Training Accuracy: %.2f%%" % (train*100))
print("Validation Accuracy: %.2f%%" % (acc*100))

Score: 0.73
Training Accuracy: 50.18%
Validation Accuracy: 49.45%
